In [23]:
import sys
import tensorflow as tf
from tensorflow import keras
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow

from pprint import pprint

checkpoint_path = "training_checkpoints/checkpoint.ckpt"

def load_test_image():
    image_str = tf.read_file("../predict-data/seetha-white.jpeg")
    image = tf.image.decode_jpeg(image_str)

    image = tf.image.resize_images(image, [28, 28])
    image = tf.image.rgb_to_grayscale(image)

    image = 1 - image
    
    return image

def convolute(images, kernel):
    kernel = tf.constant(kernel, dtype=tf.float32)
    
    kernel = tf.expand_dims(kernel, 2)
    kernel = tf.expand_dims(kernel, 3)

    #images = tf.expand_dims(images, 0)
    
    processed = tf.nn.convolution(images, kernel, padding="VALID")
    processed = tf.clip_by_value(processed, 0, 255)
    
    return processed

IDENTITY_KERNEL = [[1, 1, 1], [1, 1, 1], [1, 1, 1]]
EDGE_KERNEL = [[-1, -1, -1], [-1, 8, -1], [-1, -1, -1]]
BLUR_KERNEL = [[1, 2, 1], [2, 4, 2], [1, 2, 1]]
HV_EDGE_KERNEL = [[0, 1, 0], [1, -4, 1], [0, 1, 0]]
DIAGONAL_EDGE_KERNEL = [[1, 0, -1], [0, 0, 0], [-1, 0, 1]]

def apply_filters(images):
    edged = convolute(images, EDGE_KERNEL)
    blured = convolute(images, BLUR_KERNEL)
    #hv_edged = convolute(images, HV_EDGE_KERNEL)
    #diagonal_edged = convolute(images, DIAGONAL_EDGE_KERNEL)

    
    #cropped_images = convolute(images, IDENTITY_KERNEL)

    
    """
    return tf.concat([cropped_images, edged, blured, hv_edged, diagonal_edged],
                     3,
                     name='concat')
    """
    return tf.concat([edged, blured], 3)

#image = load_test_image()

"""
def create_model():
    images = keras.Input(shape=(28, 28, 1), batch_size=1000)
    
    #filters = keras.layers.Lambda(apply_filters)(images)
        
    #flatten = keras.layers.Flatten(input_shape=(28, 28))(images)

    relu = keras.layers.Dense(128,
                              #input_shape=(26, 26, 5),
                              activation=tf.nn.relu,
                              name='relu')(images)
    
    softmax = keras.layers.Dense(10,
                                 #input_shape=(3380,),
                                 activation=tf.nn.softmax,
                                 name='softmax')(relu)
    
    return keras.Model(inputs=images, outputs=softmax)
"""
    
model = keras.Sequential([
    keras.layers.Lambda(apply_filters),
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax),
])

# Train
# ... Load training set
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

with tf.Session():
    train_images = tf.expand_dims(train_images, 3).eval()
    test_images = tf.expand_dims(test_images, 3).eval()

    
# ... Make save weights
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                             save_weights_only=True,
                             verbose=1,
                             period=1)

# ... Compile model
model.compile(optimizer=tf.train.AdamOptimizer(),
          loss='sparse_categorical_crossentropy',
          metrics=['accuracy'])

# ... Fit model
model.fit(train_images, train_labels, epochs=15, steps_per_epoch=10, callbacks=[checkpoint_callback])

# ... Test model
test_loss, test_acc = model.evaluate(test_images, test_labels)

print("loss: {}, acc: {}".format(test_loss, test_acc))

Epoch 1/15
 9/10 [==========================>...] - ETA: 0s - loss: 5.1343 - acc: 0.3223
Epoch 00001: saving model to training_checkpoints/checkpoint.ckpt
10/10 [==============================] - 5s 525ms/step - loss: 4.8071 - acc: 0.3464
Epoch 2/15
 9/10 [==========================>...] - ETA: 0s - loss: 1.2225 - acc: 0.6707
Epoch 00002: saving model to training_checkpoints/checkpoint.ckpt
10/10 [==============================] - 4s 430ms/step - loss: 1.2022 - acc: 0.6754
Epoch 3/15
 9/10 [==========================>...] - ETA: 0s - loss: 0.8374 - acc: 0.7633
Epoch 00003: saving model to training_checkpoints/checkpoint.ckpt
10/10 [==============================] - 4s 412ms/step - loss: 0.8285 - acc: 0.7652
Epoch 4/15
 9/10 [==========================>...] - ETA: 0s - loss: 0.6711 - acc: 0.7986
Epoch 00004: saving model to training_checkpoints/checkpoint.ckpt
10/10 [==============================] - 5s 468ms/step - loss: 0.6664 - acc: 0.7995
Epoch 5/15
 9/10 [==========================